.https://taeoh-kim.github.io/blog/generative-models-part-2-improvedganinfoganebgan/


### 3. Toward Convergent GAN Training

#### 3.1 Feature matching

Feature matching은 discriminator가generator에게  overfitting을 방지하는 새로운 목적을 정해서 GAN의 불안정성을 해결한다. discriminator의 목적함수를 직접적으로 maximize하는거 대신에, 새로운 목적함수는 generator가 real data의 통계량과 일치하는 data를 만들것을 원한다. 여기서 discriminator를 사용해 가치있는 matching의 통계량을 지정한다. 특히, discriminator의 intermediate layer(중간 layer)에서의 feature들의 예상 값과 match되는 generator를 학습한다. 이런 방법은 discriminator가 생성된 data와 실제 data를 가장 잘 구분하는 특징을 찾아서 discriminator를 훈련하기 때문에 generator에는 통계적으로 자연스러운 선택이다.(생성된 데이터와 실제 데이터의 특징을 찾아서 다른 부분을 generator가 학습하도록 도와준다.)

discriminator의 intermediate layer의 activation을 나타내는 $f(x)$라 하고, generator의 새로운 목적함수는 다음과 같이 정의된다.

$$
||E_{x\sim p_{data}}f(x) - E_{z\sim p_z(z)}f(G(z))||^{2}_2
$$

Discriminator와 $f(x)$는 일반적인 방법으로 훈련된다. regular GAN의 training과 동일하게, generator가 training data(real data)의 distribution과 정확히 일치하는 fixed point를 목적함수가 가진다. 이 fixed point에 도달할 수 있다는 보장은 없다. 하지만 경험적 결과로는 feature matching이 regular feature matching은 GAN이 불안정하게 되는 상황을 벗어나는데 효과가 있음을 나타낸다.

#### 3.2 Minibatch discriminator 

GAN의 주요 실패원인은 generator가 항상 같은 point를 내보내도록 parameter를 설정되어 실패하는 것이다. discriminator의 output이 하나이기 때문에, discriminator의 gradient들이 많은 point에서 비슷한 방향을 가질 수 있다. Discriminator가 독립적으로  각 example을 처리하기 때문에, gradient들을 조율할 수 없어서 generator의 출력을 서로 다르게 할 수 있는 방법이 없다.(gradient가 비슷하게 변해서 generator가 내보내는 output이 비슷하게 된다.) 대신 모든 output들은 discriminator가 real인지 fake인지 구별하는 single point로 향하게 된다. 위에 말한 실패가 발생하면, discriminator는 generator로 부터 온 single point를 학습한다. 그러나 gradient는 동일한 output을 분리할 수 없다. 그 때 discriminator의 gradient는 generator가 만든 single point로 영원히 밀어넣는다(어떤 특정한 한 값만 출력되도록 gradient가 향한다.). 그리고 알고리즘은 원하는 distribution으로 수렴하지 못하게 된다. 이 문제를 피하기 위한 확실한 방법은 discriminator가 다양한 example data의 조합을 볼 수 있게 하고 여기서 언급되는 minibatch discrimination이라 불리는 방법을 수행한다.(minibatch를 해라.)

minibatch discrimination의 개념은 꽤 평범하다. 다양한 example을 보는 discriminator model은 generator의 collapse를 막는데 도움이 된다. 사실 이러한 측면에서 discriminator의 batch normalization의 성공은 이미 잘 설명돼있다. 그러나 지금 여기에서는 generator의 sample들이 가까워 지는 목적을 둔 model을 제한해왔다(위에 언급한 collpase이 일어나는 model을 제한해왔다.). minibatch의 example 사이에서 근접성을 modeling하기 위한 하나의 방법은 다음과 같다. $f(x_i) \in R^{A}$은 input $x_i$에서 discriminator의 어떤 intermediate layer에서 생성된 한 featur들의 vector를 의미한다. 여기에서 vector $f(x_i)$를 $T \in R^{A*B*C}$라 불리는 텐서와 곱하고 이것은 결과으로 $M_i \in R^{B*C}$라는 매트릭스가 된다. 여기서 sample $i \in {1,2,3,...,n}$에 걸쳐 $M_i$의 행들 사이의 $L_1$-distance를 계산한다. 그리고  $c_b(x_i,x_j) = exp(-||M_{i,b}-M_{j,b}||_{L_1}) \in R $인 negative exponential을 적용한다. sample $x_i$에 대한 minibatch layer의 결과값인 $o(x_i)$는 모든 sample들의 $c_b(x_i,x_j)$의 합으로 합으로 정의된다.

$$
o(x_b)_b = \sum_{j=1}^{n}c_b(x_i,x_j) \in R \\
o(x_i) = [o(x_i)_1,o(x_i)_2,...,o(x_i)_B] \in R^B\\
o(X) \in R^{n*B}
$$

다음으로, minibatch layer의 output인 $o(x_i)$를 intermediate layer의 input값인 $f(x_i)$에 concatenate하고 그 결과를 disciriminator의 다음 layer로 feed한다. minibatch feature를 generator data와 training data 따로 계산한다. 여전히 discriminator는 real data인지 fake data인지 구별해내는 것이지만 이제는 minibatch의 다른 example을 부가적인 정보로 사용할 수 있다. Minibatch discriminator는 시각적으로 괜찮은 sample을 매우 빠르게 생성할 수 있으며 feature matching보다 뛰어나다. 하지만, 흥미롭게도 feature matching은 semi supervised learning에서의 사용은 minibatch discriminator보다 뛰어나다.

<img src="../improvedganimage/improvedgan1.png">

